# Model Analysis with visualization

In [53]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

### File path

In [35]:
test_path = "../data/Test"
input_parameters = "model_parameters_efficientnetb0_224/best_test_acc/model_weights.pth"

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load model

In [37]:
from torchvision.models import efficientnet_v2_s, efficientnet_b0
model = torchvision.models.efficientnet_b0(weights=None)

batch_size = 16
img_size = 224 # 224 for efficientnet_b0, 384 for efficientnet_v2_s

### Load test data

In [38]:
# Precomputed mean and std
# for EfficientNetB0 and batch=16
# mean, std = calculate_mean_std()
mean = [0.7505297064781189, 0.5858979821205139, 0.5854080319404602]
std = [0.1179748997092247, 0.14000017940998077, 0.15512846410274506]

transform_test = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
    ])

test_dataset = datasets.ImageFolder(root=test_path, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [39]:
class_counts = [0] * len(test_dataset.classes)
for _, label in test_dataset.samples:
    class_counts[label] += 1

print(f"Total Classes: {len(test_dataset.classes)}")
print(f"Class counts: {class_counts}")
print(f"Classes: {test_dataset.classes}")

Total Classes: 9
Class counts: [16, 16, 16, 16, 16, 16, 3, 16, 3]
Classes: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [40]:
# Precomputed class weights of training set
train_class_weights = [2.1823, 0.6616, 2.6187, 0.5680, 0.6969, 0.5385, 3.2309, 1.3745, 1.7898]
class_weights = torch.tensor(train_class_weights, dtype=torch.float32).to(device)
loss_function = nn.CrossEntropyLoss(weight=class_weights)

### Configure classifier and load weights

In [41]:
num_classes = len(test_dataset.classes)
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    
    nn.Linear(num_features, 256),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(),

    nn.Linear(256, num_classes),
)

In [42]:
checkpoint = torch.load(input_parameters, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

### Evaluation

In [54]:
model = model.to(device)
model.eval()

test_loss = 0.0
correct_test = 0
total_test = 0

arr_predicted = []
arr_labels = []
arr_outputs = []

test_bar = tqdm(test_loader, desc=f"Testing:", leave=False)
with torch.no_grad():
    for images, labels in test_bar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

        # Collect predictions and true labels
        arr_predicted.extend(predicted.cpu().numpy())
        arr_labels.extend(labels.cpu().numpy())
        arr_outputs.extend(F.softmax(outputs, dim=1).cpu().numpy())  # Probabilities for AUC-ROC

        # Update progress bar
        test_bar.set_postfix({
            'loss': f"{test_loss / total_test:.4f}",
            'acc': f"{100. * correct_test / total_test:.2f}%"
        })
    
# Print test loss and accuracy
test_acc = 100. * correct_test / total_test
print(f"Test Loss: {test_loss / total_test:.4f}, Test Acc: {test_acc:.2f}%")

Test Loss: 0.9635, Test Acc: 74.58%


### Model metrics

In [ ]:
# Accuracy: The proportion of correctly classified samples.
# Precision: The proportion of true positives out of all predicted positives.
# Recall: The proportion of true positives out of all actual positives.
# F1-Score: The harmonic mean of precision and recall.
# AUC-ROC: Measures the ability of the model to distinguish between classes. 
#          For multi-class, it uses the One-vs-Rest (OvR) strategy.

In [56]:
# Convert to numpy arrays
arr_labels = np.array(arr_labels)
arr_predicted = np.array(arr_predicted)
arr_outputs = np.array(arr_outputs)

# Calculate metrics
accuracy = accuracy_score(arr_labels, arr_predicted)
precision = precision_score(arr_labels, arr_predicted, average='weighted', zero_division=0)
recall = recall_score(arr_labels, arr_predicted, average='weighted', zero_division=0)
f1 = f1_score(arr_labels, arr_predicted, average='weighted')
# AUC-ROC (One-vs-Rest for multi-class)
auc_roc = roc_auc_score(arr_labels, arr_outputs, multi_class='ovr')

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")

Accuracy: 0.7458
Precision: 0.7740
Recall: 0.7458
F1-Score: 0.7322
AUC-ROC: 0.9551
